In [1]:
import pandas as pd
import numpy as np
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv,find_dotenv
from langchain.prompts import ChatPromptTemplate,SystemMessagePromptTemplate,HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI

from operator import itemgetter
from langchain_core.tools import tool
from langchain_core.runnables import RunnablePassthrough,RunnableLambda
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent

C:\Users\MSI 1\.conda\envs\oaivenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv(find_dotenv("D:\LLM Courses\Master Langchain Udemy\.env"))

True

<h3> Creating a Tool </h3>

In [3]:
@tool
def multiply(firstInt:int, secondInt: int) -> int:
    """
        Multiply two integers togather
    """
    return firstInt*secondInt

@tool
def add(firstInt:int, secondInt: int) -> int:
    """
        Add two integers togather
    """
    return firstInt+secondInt

In [4]:
type(multiply)

langchain_core.tools.structured.StructuredTool

In [5]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two integers togather
{'firstInt': {'title': 'Firstint', 'type': 'integer'}, 'secondInt': {'title': 'Secondint', 'type': 'integer'}}


In [6]:
multiply.invoke(input={"firstInt":6,"secondInt":3})

18

<h3>Chains</h3>

In [7]:
llm=ChatGoogleGenerativeAI(model="gemini-1.5-flash")
# llm=ChatOpenAI(model="gpt-3.5-turbo")
llm

ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000002A42D684290>, async_client=<google.ai.generativelanguage_v1beta.services.generative_service.async_client.GenerativeServiceAsyncClient object at 0x000002A42D705950>, default_metadata=())

In [8]:
llmWithTools=llm.bind_tools(tools=[multiply,add])
llmWithTools

Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring


RunnableBinding(bound=ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000002A42D684290>, async_client=<google.ai.generativelanguage_v1beta.services.generative_service.async_client.GenerativeServiceAsyncClient object at 0x000002A42D705950>, default_metadata=()), kwargs={'tools': [{'function_declarations': [{'name': 'multiply', 'description': 'Multiply two integers togather', 'parameters': {'type_': 6, 'description': 'Multiply two integers togather', 'properties': {'secondInt': {'type_': 3, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'properties': {}, 'required': []}, 'firstInt': {'type_': 3, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'properties': {}, 'required': []}}, 'required': ['firstInt', 'secondInt'], 'format_': '', 'nullable': False, 'enum': []}}, {'name': 'add', 'description':

In [9]:
question="What's 5 plus Forty Two?"
response=llmWithTools.invoke(input=question)

In [26]:
response

AIMessage(content='', additional_kwargs={'function_call': {'name': 'add', 'arguments': '{"secondInt": 42.0, "firstInt": 5.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-57cc1c8b-aa29-4492-8430-9ebfda284b4d-0', tool_calls=[{'name': 'add', 'args': {'secondInt': 42.0, 'firstInt': 5.0}, 'id': 'a7acf665-22b7-4544-b91c-c7eafa099776', 'type': 'tool_call'}], usage_metadata={'input_tokens': 110, 'output_tokens': 21, 'total_tokens': 131})

In [10]:
response.content

''

In [11]:
response.tool_calls

[{'name': 'add',
  'args': {'secondInt': 42.0, 'firstInt': 5.0},
  'id': 'a7acf665-22b7-4544-b91c-c7eafa099776',
  'type': 'tool_call'}]

In [12]:
add.invoke(response.tool_calls[0]['args'])

47

In [13]:
(llmWithTools|RunnableLambda(lambda k : k.tool_calls[0]["args"])| add).invoke(input=question)

47

<h3>Agents</h3>

In [14]:
prompt=hub.pull(owner_repo_commit="hwchase17/openai-tools-agent")

C:\Users\MSI 1\.conda\envs\oaivenv\Lib\site-packages\langsmith\client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [15]:
prompt.pretty_print()

================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


In [16]:
prompt.input_variables,prompt.optional_variables

(['agent_scratchpad', 'input'], ['chat_history'])

In [17]:
@tool 
def exponentiate (base:int, exponent:int) -> int:
    """
        Exponentiate the base to the exponent power
    """
    return base**exponent   

In [18]:
tools=[multiply, add, exponentiate]

In [19]:
agent=create_tool_calling_agent(llm=llm, tools=tools, prompt=prompt)

Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring


In [20]:
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMes

In [21]:
agentExecutor=AgentExecutor(tools=tools,agent=agent, verbose=True)

In [25]:
agentExecutor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMe

In [22]:
question="Take 3 to the fifth power and Multiply that by the sum of twelve and three, then square the whole result"

In [23]:
agentExecutor.invoke(input={
    "input": question
})



> Entering new AgentExecutor chain...

Invoking: `exponentiate` with `{'exponent': 5.0, 'base': 3.0}`


243
Invoking: `add` with `{'secondInt': 3.0, 'firstInt': 12.0}`


15
Invoking: `multiply` with `{'secondInt': 15.0, 'firstInt': 243.0}`


3645
Invoking: `exponentiate` with `{'exponent': 2.0, 'base': 3645.0}`


13286025The answer is 13286025. 


> Finished chain.


{'input': 'Take 3 to the fifth power and Multiply that by the sum of twelve and three, then square the whole result',
 'output': 'The answer is 13286025. \n'}